# 9. Queries - q-sql

some text here, maybe telatex can parse this

\l data/mysp.q

In [ ]:
mktrades:{[tickers; sz]
    dt:2015.01.01+sz?31;
    tm:sz?24:00:00.000;
    sym:sz?tickers;
    qty:10*1+sz?1000;
    px:90.0+(sz?2001)%100;
    t:([] dt; tm; sym; qty; px);
    t:`dt`tm xasc t;
    t:update px:6*px from t where sym=`goog;
    t:update px:2*px from t where sym=`ibm;
    t}

In [ ]:
trades:mktrades[`aapl`googl`ibm`fb`tmobil`amazn`ms; 10]

In [ ]:
trades

- [The execution order of sql queries](https://blog.jooq.org/2016/12/09/a-beginners-guide-to-the-true-order-of-sql-operations/):
    - FROM (all the join clauses belong here)
    - WHERE
    - GROUP BY
    - HAVING
    - WINDOW
    - SELECT
    - DISTINCT
    - UNION, INTERSECT, EXCEPT
    - ORDER BY
    - OFFSET
    - LIMIT, FETCH, TOP

## 9.1. Inserting records

- Syntax rules for amend (,:):
    - pass-by-value only: it can be used only with global tables
    - despite pbv: in-place modification
    - accepts only full list syntax
- Syntax rules for insert:
    - pass-by-name only
    - in-place modification
    - accepts only full list syntax

In [ ]:
s

In [ ]:
s,:(`s6; `kincsei; 40; `budapest)

In [ ]:
s

In [ ]:
`s insert (`s7; `kincsei; 40; `budapest)

In [ ]:
delete from `s where name=`kincsei

In [ ]:
s

### 9.1.2. Bulk columnar insert

- You can insert multiple records but only as a list of __COLUMNS__!!

In [ ]:
`s insert ((`s6;`s7);(`bincsei; `kincsei);(11; 40);(`colcata; `budapest))

In [ ]:
s

### 9.1.4. Insert and foreign keys

- Defining a foreign key in a table. Syntax:
    - ([] primaryKeyColumnNameInReferencedTable:`referencedTableNameAsSymbol$values list; other columns)
- After this, you can insert only keys that referenced primary key columns contains

In [ ]:
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
tdetails:([] eid:`kt$1003 1002 1001 1002 1001; sc:126 36 92 39 98)

In [ ]:
kt
tdetails

In [ ]:
`tdetails insert 1001 133

In [ ]:
tdetails

### 9.1.5. Insert into keyed tables

- You can only insert a row only if the table does not contain the would-be-inserted value

## 9.2. Upsert

- Upsert:
    - Inserts if private key not among existing private keys
    - Updates if private key already among existing private keys
- Reference type both: it can be used with global, local and anonymous tables
    - table pass-by-value: view, does not modify table
    - table pass-by-name: in-place modification
- For bulk insert, user rows instead of columns

In [ ]:
s upsert (`s19; `pozsi; 111; `harkany)

- You can upsert into serialized and splayed tables:
    - by using a file handle instead of table name

## 9.3. The select template

__NOTES IN MS & ON PAPER__

In [ ]:
aaa:3?`3

In [ ]:
aaa

In [ ]:
mytb:([id:100+til 10] name:10?`5;iq:10?80+til 40)

In [ ]:
mytb

In [ ]:
select[(2;0W)] from mytb

### 9.3.3. Filtering with where

In [ ]:
select from mytb where[like[name;"*oo*"]] / in SQL: where name like '*dd*'

- i is like serial in SQL. you can use it to select a range of records from a table based on their index

In [ ]:
select from mytb where i within 3 5 / selecting a range based on implicit index of table

#### 9.3.3.3. multiple where subhrases

- subphrases are sequenced from left-to-right, their order affects the actual processing.
- the net effect is a progressively narrowed sublist of rows to consider.
- Consequently, there is often an optimal order that significantly narrows in the first one or few subphrases -> reduces the amount of processing.
    - Place the most limiting where subphrase first, followed by others in decreasing strictness

In [ ]:
select from mytb where[like[name;"*j*"]&iq>100]

In [ ]:
select from mytb where[like[name;"*j*"]|iq>100]

#### __Nested columns in where__

- Use one of the maps iterators: each('), each left(\:), each right (/:), each parallel, each prior (':), case ('                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  )
    - cross: each left and each right combined \:/:

In [ ]:
t:([] f:1.1 2.2 3.3; s:("abc";enlist "d";"ef"))

In [ ]:
select from t where s="ef" / will fail

In [ ]:
select from t where s~"ef"

In [ ]:
select from t where s~\:"ef"

In [ ]:
select from t where "ef"~/:s

In [ ]:
1 2 3 in'(1 0 1;til 100;5 6 7)

#### __9.3.3.5. fby in where__

- fby is the equivalent of HAVING in SQL
- fby returns the vaue of the aggregate across each group

- syntax: (f_agg;expr_col) fby colname
    - f_agg: aggregate function
    - expr_col: column expression on which f_agg is applied
    - colname: is the column to be grouped

- __what fby does__: Since it returns the value of the aggregate across each group, you simply compare the target column to the fby result to get a boolean vector with 1b at precisely the records whose fields match the aggregate on the group.

In [ ]:
p

In [ ]:
select from p where weight=max weight

In [ ]:
select max weight by city from p

In [ ]:
select name, color, max weight by city from p

In [ ]:
select first name, first color, max weight by city from p

In [ ]:
select from p where weight=(max;weight) fby city

In [ ]:
select from p where weight=fby[(max;weight);city]

- The right solution with multiple where phrases:

In [ ]:
select from p where weight=fby[(max;weight);city],color=`blue

- __Grouping on multiple columns__
    - encapsulate multiple columns in an anonymous table

In [ ]:
select from p where weight=fby[(max;weight);([] city;color)]

### __9.3.4. Grouping and aggregation__

- grouping and aggregation is performed together in SQL
- grouping and aggregation is performed independently in q-sql

#### 9.3.4.1. Aggregation without grouping

- aggregation by one column -> result atom
- aggregation by multiple columns -> result a table with a single record

In [ ]:
sp

In [ ]:
select sum qty from sp

In [ ]:
select total:sum qty, mean:avg qty from sp

#### 9.3.4.2. Grouping without aggregation

- similar to GROUP BY in SQL
- the _by_ phrase groups rows having common values in specified columns
- the result of a by phrase query is a __keyed table__ whise key columns are in the by phrase
- the by phrase is a generalization of the _group_ function on lists
- result: nested columns
- not advised to do this. nested columns are:
    - slower
    - more cumbersome to work with (need adverbs)
    - unnecessary
- _ungroup_ reverses the nested result of grouping without aggregation (cannot be used on lists groupped with _group_)

In [ ]:
s

In [ ]:
select by s from sp / by not specifying any non-group by columns, the last elements of these columns are selected

In [ ]:
spGrouped:select p,qty by s from sp

In [ ]:
select p,qty by s from sp

In [ ]:
`s xgroup sp / is the same as the above select phrase except it is not ordered by the group by columns

In [ ]:
select p,total:sum qty by s from sp

In [ ]:
ungroup spGrouped

In [ ]:
groupped:group 1 3 2 5 7 1 3 5 7 2 1

#### 9.3.4.3. Grouping with aggregation

- usual usage of _by_
- use case: aggregate col1 by col2 in _the by_ phrase
- result: a keyed table whose keys are the values of the grouped _by_ column
- grouping on multiple columns -> a table with composite keys

In [ ]:
select total:sum qty by s from sp

In [ ]:
select total:sum qty by s,p from sp

- you can group on computed columns (a _by_ subphrase can contain a q expression)
    - and use the computed column as a new column in the query
- a frequent expression is xbar which can be used to create intervals from a list containing numeric atoms

In [ ]:
trade:([] c1:00:00:00.000+til 1000000;c2:1000000?`a`b;c3:1000000?100.)

In [ ]:
trade

In [ ]:
select avg c3 by 100 xbar c1 from trade

## 9.4. __The _exec_ template__

- the syntax of the exed template is identical to that of select:
    - exec <p_s> by <p_b> from t_exp where <p_w>
- the selut type of exec depends on the number of columns in its select phrase
    - one columns yields a list
    - two or more columns yields a dictionary
- the resulting columns lists do not have to be rectangular!!!
- when selecting only one column with exec, you can still place constraints on other columns

## 9.5. The update template

- syntax (identical to select): update <p_u> <by p_b> from t_exp where <p_w>

- upsert semantics:
    - if the column exists, it is updated (you can omit the column names)
    - if it does not, it is inserted
- column can be upserted with specifying only one scalar
- If you __add a new column__ in update with a where phrase, the fields in the non-selected rows will have the __null value__ of the appropriate type.

In [ ]:
t:([] c1:`a`b`c; c2:10 20 30)

In [ ]:
t

In [ ]:
update c3:42 from t / table is passed by value -> it is not modified

In [ ]:
update c3:42 from `t / table is passed by name -> it is modified in-place

In [ ]:
update c3+10 from t

In [ ]:
update c4:10 from `t where c2>20

In [ ]:
update c5:101 from `t where c1=`c

In [ ]:
update c5:303 from `t where null c5

### 9.5.2. Update-by

In [ ]:
update avg c5, avg c4 by c1 from t

In [ ]:
update cumqty:sums qty by s from sp / update with aggreagate and uniform functions updates all fields of the column

## 9.6. The delete template

- reference: both pass-by-value and pass-by-name can be used
- both rows and columns can be deleted
- syntax: simplified select
    - delete <p_cols> from t_exp <where p_w>
- pass by name have to be used to modify table in-place

- use __delete instead of select__ if you want to select a lot of columns
    - but use pass by value!!!

In [ ]:
delete c5 from t / deletes column from table

In [ ]:
delete from t where c2=40 / deletes row from table

## 9.7. Sorting

- Tables are ordered in q tables by the time of their insertion into the table
- they can be reordered by sorting in the select template -> there are no order by clause in q-sql
- syntax: `colName1 `colname2 xasc (xdesc) tableName (use pbn to modify in-place)
- mixed sort: colName1 xasc colname2 xdesc tableName

In [ ]:
exec c4 from t where c1=`a

In [ ]:
update c4:33,c5:44 from `t where c4=0N and c5=0N

In [ ]:
`c2`c5 xdesc t

In [ ]:
`c1 xdesc `c4 xasc t / xasc and xdesc are stable sorts, which can be used in mixed sorts

## 9.8. Renaming and rearranging columns

- Syntax:
    - `newColName xcol tableName: renames first column in tableName
- Tables can only be passed by value -> to modify table, reassign it

- cols y: get column names of a table as a symbol list, where y is a table
- x xcol y: rename column names in a table, where x is a symbol scalar or list
- x xcols y: rearrange columns of a table, where x is a symbol list of column names in y
    - column names in x are ordered at the beginning of the table, not included columns left untouched at the end of the table
    - y cannot be a keyed table
    - only pbv -> reassign to modify table

In [ ]:
reTable:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

In [ ]:
reTable

In [ ]:
`newCol`new2 xcol reTable

- To __rename columns__ in the middle of the table: xcol

In [ ]:
@[cols[t]; where cols[t]=`c2; :; `new2] xcol t

In [ ]:
t

In [ ]:
@[cols[t]; where cols[t] in `c1`c4; :; `new1`new3] xcol t

- To rearrange columns: xcols

## 9.9 Joins

- __IN SQL__:
    - Relational structure is static: primary and foreign keys are defined in DDL before data can be used
- __IN Q__:
    - You can define tables and relations both statically and dynamically
    - You can join tables that do not have a relation predefined
    - left and right join is achieved in q by reversing the operands in the join
    - Equijoins: the primary and the foreign keys must be equal
    - Non-equijoin (as of join): less-than-or-equal to the primary keys in another table

### 9.9.1. Implicit joins

- __implicit left join__: you can retrieve columns from the implicitly joined table by using dot notation on the foreign key column of the base table
- __implicit right join__: does not work here, because the table with the primary key does not know which tables a related to it

In [ ]:
sp

In [ ]:
p

In [ ]:
select sname:s.name, p.color, qty from sp

In [ ]:
select s, name from s

- Multiple joins: you can reference a table related to your table only through the table with the primary key

In [ ]:
emaster:([eid:1001 1002 1003 1004 1005] currency:`gbp`eur`eur`gbp`eur)

In [ ]:
emaster

In [ ]:
s
p
sp
emaster

In [ ]:
update eid:`emaster$1001 1002 1005 1004 1003 1001 1002 1005 1004 from `s

In [ ]:
s

In [ ]:
select sname:s.name, s.eid.currency, qty from sp

### 9.9. 2. __Ad hoc left join (lj)__:
- dynamically joins two tables regardless of a foreign relation being present
    - if there is a relation, it is used
    - if there isn't, the linkage is constructed dynamically
- use case: if you want to join the two tables only once or twice
- syntax: sourceTable jl targetTable
    - result: all the columns and records of the source table augmented by the matching columns and records of the target table:
        - augmented values in the source fields contain null values
-  __when the source table has a foreign key__, an ad-hoc left join is equivalent to listing all columns from both tables in an implicit join.
- When the tables source and target have duplicate non-key columns, the operation has upsert semantics.
    - The values in the right operand (target) columns prevail over those in the left operand (source).
    - In SQL, the result contains both columns with suffixes to ensure unique names.

### 9.9.3. Column lookup

- Joins two tables where there are no matching column names by renaming the column names in the target columns
- Method:
    -  Choose the fields from the target table by indexing while renaming the common column by defining an anonymous table with the matching column name and assigning the old values from the target table

In [ ]:
t:([] k:1 2 3 4; c:10 20 30 40)
kt:([k1:2 3 4 5]; v:2.2 3.3 4.4 5.5)

In [ ]:
t
kt

In [ ]:
kt[2;`v]

In [ ]:
select c, v:kt[([] k1:k); `v] from t

In [ ]:
select c,v from t lj `k xkey select k:k1,v from kt / same thing with left join

### 9.9.4. __Ad hoc inner join (ij)__:

- The two inner joined table do not have to have a relation defined by primary/foreign keys (same as lj)
- x ij y where x is the source (keyed) table and y is the target (keyed) table
    - they have pr/fg keys or
    - have columns that are exactly matching (same column name, type and values)
- For duplicate columns, upsert semantics holds (target column values prevail when non-key columns match)

### 9.9.5. Equijoin (ej)

- ej corresponds to an SQL inner join
- triadic function
- syntax: ej[commonColumn;sourceTable;targetTable]

- x xkey y sets column x as primary key of table y

In [ ]:
t1:([] k:1 2 3 4; c:10 20 30 40)
t2:([] k:2 2 3 4 5; c:200 222 300 400 500; v:2.2 22.22 3.3 4.4 5.5)

In [ ]:
t1 ij `k xkey t2 / sets k column as primary key then inner joins it with source table

In [ ]:
ej[`k;t2;t1] / inner joins the two tables on column k

In [ ]:
ej[`k;t1;t2]

### 9.9.6. Plus join (pj)

- is a lj that replaces upsert semantics for duplicate column names with __addition__
- two tables must have the identical schemas
- all non-key columns must be numeric
- missing and null values are treated as zeros

### 9.9.6. Unoin join (uj)

- In q, the equivalent of a SQL union is x,y where:
    - it joins two tables with matching schemas
    - and the result is the sum of records in the two tables
- The __uj__ in q, however, is more powerful:
    - uj vertically combines __any__ two (keyes) tables
    - x uj y:
        - records for common columns in y are appended to x
        - records for unmatched columns are also appended, but missing fields are filled with zero values

__The operation of uj is expensive__ so it should only be used when necessary; in particular, when the tables conform, use , or raze. Be especially cautious when low latency is paramount.

In [ ]:
t1:([] c1:`a`b`c; c2: 10 20 30)
t2:([] c1:`x`y; c3:8.8 9.9)

In [ ]:
t1 uj t2

- union join can be applied to more than two tables with the over adverb

In [ ]:
t3:([] c1:`e`f`g; c2:50 60 70; c3:5.5 6.6 7.7)

In [ ]:
(uj/) (t1; t2; t3)

### 9.9.8. As-of joins (aj, aj0, asof)

- As-of joins are so-named because they most often join tables along time columns to obtain a value in one table that is __current as of a time in another table__
- non-equijoin: tney match on less-than-or-equal
- however, it is mostly used on time values, aj are not restricted to time values
    - it can be used on non-numeric data. e.g.: what is the job title of a person as of now
- syntax: aj[`c1...`c2;t_1;t_2] where:
    - first operand is a simple list of symbols referencing common column names in the tables to be joined
    - second operand: source table
    - third operand: target table
- Characteristics:
    - Common columns do not have to be keys, but
        - aj on keys are __faser__
    - returns all columns in both tables
- Semantics:
    - The match on all specified columns is by equals,
    - except the last, which is less-than-or-equal: Assuming the records are sequenced by cn in both tables, for a given cn value in t1, the match picks __the greatest cn in t2 less than or equal to the given value in t1__

- The canonical example for aj is matching trades with quotes.
- To know if a trade represents best execution, you want to compare the trade price to the current quote – i.e., the most recent quote up to and including the time of the trade.
- You clearly want an __equi-join on the date and symbol, but a non-equijoin on the time__ when matching the trades and quotes.

In [ ]:
show t:([] ti:10:01:01 10:01:03 10:01:04;sym:`msft`ibm`ge;qty:100 200 150)

In [ ]:
show q:([] ti:10:01:00 10:01:01 10:01:01 10:01:02;sym:`ibm`msft`msft`ibm;px:100 99 101 98)

In [ ]:
aj[enlist `ti;t;q]

In [ ]:
aj[`ti`sym;t;q]

In [ ]:
aj[`sym`ti;t;q]

- while aj joins the two tables on the source table's time value,
- aj0 joins the tables on the time of the target table

In [ ]:
aj0[`sym`ti;t;q]

- __asof__ performs the same match as aj, but with a table and a single record (can be a table as well) as parameters

In [ ]:
show t:([] ti:10:01:01 10:01:03 10:01:04; sym:`msft`ibm`ge; qty:100 200 150; px:45 160 55)

In [ ]:
t asof `sym`ti!(`ibm;10:01:03)

### 9.9.9. Window join (wj, wj1)

- window joins are a generalization of as-of joins
- wj a especially geared for analyzing the relationship between trades and quotes in finance:
    - wj is a built-in function that allows for looking at the behaviour of quotes in a neighborhood of each trade
- Semantics:
    - works on unkeyed tabels
- To use wj, we need to setup fixed-width windows
- Syntax:
    - wj[w;c;t;(q;(f0;c0);(f1;c1))], where
        - w is a list of windows in the form of a nested list which contains two lists: one the start times and the other the end times
        - c is a list of column names by which to join the two tables (same as first parameter of aj)
        - t is the source table
        - a list of 3 items:
            - 1. q is the target table
            - 2. a list of an aggregate function and a column name to be aggregated
            - 3. a list of another aggregate function and another column name to be aggregated
                - to wj without aggregation, use the identity function: ::

In [ ]:
show t:([]sym:3#`aapl;time:09:30:01 09:30:04 09:30:08;price:100 103 101)

In [ ]:
show q:([] sym:8#`aapl;
    time:09:30:01+(til 5),7 8 9;
    ask:101 103 103 104 104 103 102 100;
    bid:98 99 102 103 103 100 100 99)

In [ ]:
show w:-2 1+\:t `time / takes the values in the time column and adds 1 and subtracts 2 from them and creates two separate lists

In [ ]:
c:`sym`time

In [ ]:
wj[w;c;t;(q;(::;`ask);(::;`bid))]

In [ ]:
wj[w;c;t;(q;(avg;`ask);(avg;`bid))]

- __wj1__: includes current values (just) outside of the specified windows
    - wj1 comes handy when there are no values within the specified windows
- wj1 syntax is identical to wj

In [ ]:
wj1[w;c;t;(q;(::;`ask);(::;`bid))]

## 9.10. Parameterized queries

- In relational databases, stored procedures are used to save recurring database transactions through user defined custom functions.
- To be able to do this, you need to learn either
    - a proprietary query language (pl/sql, mysql, t/sql, postgresql) to place functionality close to data or
    - extract the data into an application server to perform calculations.
    - Various multi-tier architechtures have evolved to address this problem, but they increase system cost and complexity
- In q, this is a non-issue as q
    - is the stored procedure language
    - and it has the power and perfomance to process big data.
    - any q function that operates on a table is a stored procedure.
- Function parameters can be used to supply specific values for queries.
- In particular, the select, exec, update and delete templates can be invoked within a function with parameters to yield a parameterized query.
    - values and tables can be passed in as well. The latter can be passed as pbv or pbn

- Restrictions on parameterized queries with templates:
    - columns cannot be passed as parameters: they are only syntactic sugar of the template
    - multiple parameters must be declared explicitly. Recommended to __make all parameters explicit__.

In [ ]:
tpq:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

In [ ]:
select from tpq where c2>15

In [ ]:
proc:{[sc] select from tpq where c2>sc}
proc 15

In [ ]:
proc2:{[nms;sc] select from tpq where c1 in nms, c2>sc}

In [ ]:
proc2[`a`c; 15]

In [ ]:
proc3:{[sc;t] select from t where c2>sc}

In [ ]:
proc3[20; `tpq]

In [ ]:
tpq

- Workarounds to parameterize column names:
    - Slow and dangerous method: Pass the query as string with the value function
    - Fast and safe method: functional form of queries:
        - any component of a query can be passed as an argument in functional form
        - e.g.: all columns are referred to by symbolic name -> column names can be passed as symbols

In [ ]:
sd1:{[sc;table;cmn] select from table where sc<value ssr[;"p2";string table] ssr["exec p1 from p2";"p1"; string cmn]}

In [ ]:
sd1[20;`tpq;`c2]

## 9.11. Views (::)

- In SQL, a view is effectively is a query expression whose reslit set can be used like a table
- Views are used to encapsulate data: hiding columns or simplifying simple queries
- In q-sql, a view is a named table expression created as an alias with the double-colon operator
- syntax viewVariable::select template
- variable vs view:
    - a select assigned to a variable:
        - a variable is a separate copy of the table
    - a select assigned to a view is:
        - a named table expression created as an __alias__
        - it is a query expression itself
        - it is mutable: it changes when the underlying table changes
- view: get the underlying query of a view alias
- views `. lists all view aliases in the root

In [ ]:
trades

In [ ]:
show copy:select from trades where qty>7000

In [ ]:
vue::select from trades where qty>7000

In [ ]:
update qty:101 from `trades where qty>7000

In [ ]:
trades

In [ ]:
copy

In [ ]:
view `vue

In [ ]:
views `.

In [ ]:
t:([] c1:`a`b`c; c2:10 20 30)
u:select from t where c2>15
v::select from t where c2>15

In [ ]:
u

In [ ]:
v

In [ ]:
update c2:15 from `t where c1=`b

In [ ]:
u

In [ ]:
v

## 9.12. Functional forms

- Functional forms of queries are hard to grasp for the following reasons:
    - There is a high level of abstraction and generalization in them
    - There no analogies for it in SQL or other programming languages
    - Information density is maximized in functional forms of q-queries

- Advantages of the functional forms of the select, exec, update and delete templates:
    - can be parameterized
    - useful for queries that are generated or completed programmatically
- There is no performance difference between these templates and the functional form??????

- There are two functional forms:
    - One for select and exec templates: ?[t;c;b;a;n;iv;co]
    - Another one for update and delete templates: ![t;c;b;a]
    - where:
        - t -> the from phrase (table)
        - c -> the where phrase, which is a list of constraints (comparison operators, like, within, and, or)
        - b -> the by phrase, a dictionary of groupbys or a flag contolling other aspects of the query
            - 1b means select distinct in place of the group by phrase
        - a -> the select phrase, which is a dictionary of aggregates
    - c, b, a can contain columns of t and any variables in scope
        - n (limit; optional) -> select[n]: 5th parameter to the functional form; __type scalar long__
        - iv (p1 for order by; optional) -> select[>c1]: 6th parameter as the initial value of the sort
            - usually long infinity: 0W
        - co (p1 for order by; optional) -> select[>c1]: 7th parameter as the comparison operator for the sort
            - a list comprising of the comparison operator in k form and the column name for the sort (>:;`c1)

- Rules for expression interpretation:
    - Columns are always represented by their symbolic names (`col1`col2)
    - To distinquish literal symbols (or lists of symbols) appearing in expression from symbolic column names,
        - you have to enlist them

- Semantics:
    - t: the table can be pbn or any expression that returns a table or a keyed table

In [1]:
fqt:([] c1:`a`b`a`c`a`b`c; c2:10*1+til 7; c3:1.1*1+til 7)

In [ ]:
fqt

In [ ]:
fqfn:{([] c1:`a`b`a`c`a`b`c; c2:10*1+til 7; c3:1.1*1+til 7)}

In [ ]:
fqfn[]

### 9.12.1. Functional select

In [3]:
?[`fqt;();0b;()]~select from fqt / table with no constraints, no group by phrase and no aggregate phrase

1b


In [7]:
?[`fqt;();();`c1]~exec c1 from fqt // exec syntax

1b


In [ ]:
 / multiple constrainst should be in a list form
?[`fqt; ((>;`c2;35); (in;`c1;enlist `b`c)); 0b; ()]~select from fqt where c2>35,c1 in `b`c

In [ ]:
 / aggregating without grouping
?[`fqt; (); 0b; `maxc2`wtavg!((max;`c2); (wavg;`c2;`c3))]~select maxc2:max c2, wtavg:c2 wavg c3 from fqt

In [ ]:
 / grouping without aggregation: column names are enlisted because of dictionary format, not because they are symbolic literals
?[fqt; (); (enlist `c1)!enlist `c1; ()]~select by c1 from fqt

In [ ]:
fqc:((>;`c2;35); (in;`c1;enlist `b`c))
fqb:(enlist `c1)!enlist `c1
fqa:`maxc2`wtavg!((max;`c2); (wavg;`c2;`c3))

In [ ]:
 / putting it alltogether: constraints (fqc), group by (fqb), aggregation (fqa)
?[`fqt;fqc;fqb;fqa]~select maxc2:max c2, wtavg:c2 wavg c3 by c1 from fqt where c2>35,c1 in `b`c

In [ ]:
 / select distinct: put 1b in place of the group by phrase
?[`fqt; (); 1b; `c1`c2!`c1`c2]~select distinct c1,c2 from fqt

### 9.12.2. Functional exec

- it is nearly identical to functional select
    - slight variations in the parameters indicate which is intended
- c, the constraint parameter is unchanged (not discussed here)

- Semantics of exec on a single column:
    - t: the same
    - c: the same, not discussed
    - b: if there is no grouping, use () as opposed to 0b in case of select
    - a: here you specify the return type as a list or a dictionary (in select you always have to specify a dictioary as you always get a table returnd which has column dictionaries)
        - to return a list, specify the aggregate as a __parse tree__ (distinct;`c1)
        - to return a dictionary, specify a dictionary (enlist `newName2)!enlist (distinct;`c1)

- To exec on multiple columns, no grouping:
    - b:() generalempty list
    - a:(`c1`c2)!((distinct;`c1); `c2) - same as in select
- To group on a single column:
    - b: specify the symbol name in the by parameter. e.g., ?[`t;();`c1;`c2] (you do not need to enlist the column names)

In [ ]:
?[`fqt;();`c1;`c2]

### 9.12.3. Functional update

- The syntax of functional update is identical to that of functional select, except:
    - ! is used of ?
    - t can be both passed by value and name
- Using enlisting for 3 different purposes:
    - Making a list of parse trees from a single parse expression
    - Creating singleton dictionaries
    - Distinguishing literal symbols from column names

In [8]:
c:enlist (=;`c1;enlist `a)
b:0b
a:(enlist `c2)!enlist 100

In [9]:
![fqt;c;b;a]

c1 c2  c3 
----------
a  100 1.1
b  20  2.2
a  100 3.3
c  40  4.4
a  100 5.5
b  60  6.6
c  70  7.7


In [10]:
fqt

c1 c2 c3 
---------
a  10 1.1
b  20 2.2
a  30 3.3
c  40 4.4
a  50 5.5
b  60 6.6
c  70 7.7


In [ ]:
update c2:sums c2 by c1 from t
![`t; (); (enlist `c1)!enlist `c1; (enlist `c2)!enlist(sums; `c2)] / does not work

### 9.12.4. Functional delete

- The syntax of functional delete is a simplified form of functional update: ![t;c;0b;a]
    - t: table name can be passed by value and by name
        - pbv: returns the table after deletion
        - pbn: does not return anything
    - c: a list of parse trees, where subphrases are in a list; use general empty list if a is present ()
    - b: 0b (always?)
    - a: list of symbol column names or an empty list of symbols `symbol$()
- Either c or a must be present but not both:
    - you can specify either columns or rows to be deleted
    - but you cannot delete both columns and rows at the same time

In [ ]:
t:([] c1:`a`b`c`a`b; c2:10 20 30 40 50)

In [ ]:
![t;enlist (=;`c1;enlist `b);0b;`symbol$()]~delete from t where c1=`b

In [ ]:
![t;enlist (=;`c1;enlist `b);0b;`symbol$()]

In [ ]:
![t;();0b;enlist `c2]~(delete c2 from t)

In [ ]:
t

# __MY Functional query practice on trades table__

- Cheat to create functional queries in k from q-sql queries:
    - parse "q-sql query", which returns the parse tree of the q-sql query in k
- Good for reverse engineering and practice

In [ ]:
parse "select from trades"

In [ ]:
parse "select sym from trades"

In [ ]:
parse "select sym, qty, px from trades where qty>7000"

In [ ]:
parse "select qty, px by sym from trades where qty>7000"

In [ ]:
select qty, px by sym from trades where qty>7000

In [ ]:
parse "select total:sum qty, mean:avg px by sym from trades"

In [ ]:
/
Output:
?`trades()(,`sym)!,`sym`total`mean!((sum;`qty);(avg;`px))
\

In [ ]:
select from trades

In [ ]:
?[`trades;();0b;()!()] / select from trades

In [ ]:
?[`trades;();0b;(enlist `sym)!(enlist `sym)] / select sym from trades

In [ ]:
?[`trades;();0b;(`symbol`quality)!(`sym`qty)] / select symbol:sym,quality:qty from trades

In [ ]:
?[`trades;enlist (>;`qty;7000);0b;(`sym`qty`px)!(`sym`qty`px)] / select sym,qty,px from trades where qty>7000

In [ ]:
?[`trades;();(enlist `sym)!(enlist `sym);(`qty`px)!(`qty`px)] / select qty,px by sym from trades

In [ ]:
 / select totalqty:sum qty,averagepx:avg px by sym from trades
?[`trades;();(enlist `sym)!enlist `sym;(`totalqty`averagepx)!((sum;`qty);(avg;`px))]

In [ ]:
 / select totalqty:sum qty,avgpx:avg px by symbol:sym from trades where sym=`ibm
?[`trades;enlist (=;`sym;enlist `ibm);(enlist `symbol)!(enlist `sym);(`totalqty`avgpx)!((sum;`qty);(avg;`px))]

In [ ]:
/
Should reverse engineer this:
?
`trades
,,(<;7000;(k){$[(#x 1)=#y;@[(#y)#x[0]0#x 1;g;:;x[0]'x[1]g:.=y];'`length]};(en..
(,`sym)!,`sym
`qty`px!((sum;`qty);(avg;`px))

To get the equivalent of this:
select sum qty,avg px by sym from trades where 7000<(sum;qty) fby sym

\

In [ ]:
/ ?[`trades;????????;(enlist `symbol)!(enlist `sym);(`totalqty`avgpx)!((sum;`qty);(avg;`px))]

In [ ]:
parse "select sum qty,avg px by sym from trades where 7000<(sum;qty) fby sym"

### 9.13. Examples

- Takeaways:
    - you can use custom functions as aggregatros in queries

__HOMEWORK: DO THE EXERCISES__